# Assignment 1: Recommender Systems
## 1.1 Naive Approaches
Notebook for the naive approaches recommender systems. We develop a 'gloabl average rating', 'average user rating', 'average movie rating' and two linear regression models recommender systems.


In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
#read the rating data 
df = pd.read_csv('ml-1m/ratings.dat', sep = '::', names = ['UserID', 'MovieID', 'Rating', 'Timestamp'])

/var/folders/zq/kxyb9__d7q11lb0tprf805cr0000gn/T/ipykernel_7542/2921473051.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('ml-1m/ratings.dat', sep = '::', names = ['UserID', 'MovieID', 'Rating', 'Timestamp'])


There are movies in the data for which there are no ratings at all. These are of no interest for our recommender system, so we don't want to have columns of these in our utility matrix. The next cell finds those movies are stores their ID in the 'del_col' array. Note that we don't have to do this for users, for each user we have at least one rating.

In [8]:
#we are looking for movies which are completely missing in the data

y = np.amax(df['MovieID'])   #highest movie ID in the data
movie_Id = np.unique(df['MovieID'])   #all the movie ID's in the data
miss_movieId = []           #list which will store which movie ID's are missing in the data 
for i in range(1, y + 1):   #loop through all possible ID's
    if len(movie_Id[movie_Id == i]) == 0:   #if ID i is not present in the data, we want a record of this
        miss_movieId.append(i)

del_col = np.array(miss_movieId) - 1 #these are the columns to be deleted straightaway in the utility matrix, -1 is needed because indexing begins at 0

    

The following cell creates 5 sets of rating indexes from the original dataframe df (=ratings.dat). The sets are randomy picked and are of the same size (with exception of the last set which is slightly smaller) and we make sure that there is no overlap between the sets. The sets are later used to generate the 5 training and associated training sets for the 5-fold cross validation. We will store the sets in a dictionary called 'sets'. For example, for the first fold we will use set1, set2, set3 and set4 to create the training data and set5 is used for the test data. Using our method here, we ensure that each training/test set of the 5 folds will have the same number of known ratings (i.e non-zero elements in the utility matrix).

In [9]:
#generate the 5 sets for the 5-fold cross validation

np.random.seed(10)   #our seed for splitting the rating data into 5 oarts
sets = {'set1':[], 'set2':[], 'set3':[], 'set4':[], 'set5':[]}   #dictionary which will store the sets

l = len(df) // 5   #length of a set or number of known ratings in a set 
index_list = np.arange(0, len(df), 1)   #all indexes of known ratings that exist

for j in range(1, 6):  #loop for creating the 5 sets
    if j != 5:
        i = np.random.choice(len(index_list), l, replace = False)
    else:
        i = np.random.choice(len(index_list), len(df) - 4*l, replace = False)   #only the fifth set is of different length so that we use up everything
    sets['set{}'.format(j)] = index_list[i]
    index_list = np.delete(index_list, i)
    

ut_matrices is a function for generating the training utility matrix and test utility matrix

In [20]:
def ut_matrices(train_index, test_index, df, del_col, sets):
    #function for generating the training utility matrix and test utility matrix
    #train_index: indicates which 4 sets are included in the training matrix
    #test_index: indicates which set is used for the testing
    
    x, y = np.amax(df['UserID']), np.amax(df['MovieID'])
    R = np.zeros((x, y))  #training data, missing rating are defined as 0

    for j in train_index:  #we now fill the utility matrix with the training data
        U, M, RA = df['UserID'].iloc[sets['set{}'.format(j)]], df['MovieID'].iloc[sets['set{}'.format(j)]], df['Rating'].iloc[sets['set{}'.format(j)]]
        for u, m, ra in zip(U, M, RA):
            R[u - 1, m - 1] = ra #-1 is needed because the smallest ID is 1
    
    R = np.delete(R, del_col, axis = 1)  #get rid of these completely empty columns, these movies have no ratings
                
    R_test = np.zeros((x, y))   #test data, missing rating are defined as 0
    U, M, RA = df['UserID'].iloc[sets['set{}'.format(test_index)]], df['MovieID'].iloc[sets['set{}'.format(test_index)]], df['Rating'].iloc[sets['set{}'.format(test_index)]]
    for u, m, ra in zip(U, M, RA):
        R_test[u - 1, m - 1] = ra #-1 is needed because the smallest ID is 1
    
    R_test = np.delete(R_test, del_col, axis = 1) #get rid of these completely empty columns, these movies have no ratings
    
    return R, R_test   #R is the training utility matrix, R_test is the training utility matrix

#### Gloabal average recommender system

In [22]:
#global approach

indx = np.arange(1, 6, 1)  
RMSE = []  #will store the training RMSE of the 5 folds
MAS = []   #will store the testing RMSE of the 5 folds

RMSE_test = []  #will store the training RMSE of the 5 folds
MAS_test = []   #will store the testing RMSE of the 5 folds

for i in range(5):   #do a loop for the 5-fold validation
    print('=== Fold {} ==='.format(i + 1))
    train_index = np.delete(indx, i)  #indicates which data is used for the training matrix R
    test_index = i + 1                #indicates which data is used for the test matrix R_test
    R, R_test = ut_matrices(train_index, test_index, df, del_col, sets) #create the training and test utility matrix
    
    av = np.sum(R) / np.count_nonzero(R)  #the is the averge value of all the ratings in the training set
    print('Average value of the training utility matrix is:', av)
    
    #compute the rmse and mas on the training data using the global average av as the prediction
    i, j = np.where(R != 0)   #find the ratings we know
    err = np.sum((R[i, j] - av)**2)  #compute the total difference^2 of all the known ratings and their predictions
    N = len(R[i, j])
    rmse = np.sqrt(err / N)   
    print('The training RMSE is:', rmse)
    err_abs = abs(R[i, j] - av)  
    mas = np.sum(err_abs) / N  
    print('The training MAS is:', mas)
    RMSE.append(rmse)
    MAS.append(mas)
     
    #compute the rmse and mas on the test data using the global average av as the prediction
    i, j = np.where(R_test != 0)   #find the ratings we know in the test set
    err = np.sum((R_test[i, j] - av)**2)
    N = len(R_test[i, j])
    rmse = np.sqrt(err / N)
    print('The test RMSE is:', rmse)
    err_abs = abs(R_test[i, j] - av) 
    mas = np.sum(err_abs) / N
    print('The test MAS is:', mas)
    RMSE_test.append(rmse)
    MAS_test.append(mas)
    
print('==================================')
print('Average RMSE score on the training set:', sum(RMSE) / 5)
print('Average MAS score on the training set:', sum(MAS) / 5)
print('Average RMSE score on the test set:', sum(RMSE_test) / 5)
print('Average MAS score on the test set:', sum(MAS_test) / 5)

=== Fold 1 ===
Average value of the training utility matrix is: 3.5821152557962828
The training RMSE is: 1.117465183252614
The training MAS is: 0.9340616016414436
The test RMSE is: 1.1156451872976754
The test MAS is: 0.9326439144126983
=== Fold 2 ===
Average value of the training utility matrix is: 3.5818890532988075
The training RMSE is: 1.1173453351594858
The training MAS is: 0.9341219668728845
The test RMSE is: 1.1161247915519716
The test MAS is: 0.932572468547058
=== Fold 3 ===
Average value of the training utility matrix is: 3.581356665100329
The training RMSE is: 1.1168819291993668
The training MAS is: 0.9337482777544189
The test RMSE is: 1.1179783884285186
The test MAS is: 0.9344673812766995
=== Fold 4 ===
Average value of the training utility matrix is: 3.5808917627298267
The training RMSE is: 1.1171752555782735
The training MAS is: 0.9339118035944793
The test RMSE is: 1.1168063748756516
The test MAS is: 0.9341627003449025
=== Fold 5 ===
Average value of the training utility ma

#### Average movie rating recommender system

In [29]:
def average_item(R):
    #function which computes the average rating for each movie (i.e average value of each column)
    
    av = np.sum(R) / np.count_nonzero(R)   #the fallback value we use when columns are completely empty in the training set  
    #we are computing the average rating of each movie now
    clm_sums = np.sum(R, axis = 0) #we sum together colums (each column represents a movie)
    clm_n0_count =  np.count_nonzero(R, axis = 0) #number of non-zero ratings along each column
    i_miss = np.where(clm_sums == 0) #movies for which there are no ratings in the training set
    clm_sums[i_miss] = av     #for movies which went missing in the set, we use the fallback value av, i.e we assume that their average rating is the global average
    clm_n0_count[i_miss] = 1
    av_item = clm_sums / clm_n0_count  #average rating for each item
    return av_item

In [30]:
#average rating per movie approach

indx = np.arange(1, 6, 1)

RMSE = []  #will store the training RMSE of the 5 folds
MAS = []   #will store the testing RMSE of the 5 folds

RMSE_test = []  #will store the training RMSE of the 5 folds
MAS_test = []   #will store the testing RMSE of the 5 folds


for i in range(5):   #do a loop for the 5-fold validation
    print('=== Fold {} ==='.format(i + 1))
    train_index = np.delete(indx, i)  
    test_index = i + 1               
    R, R_test = ut_matrices(train_index, test_index, df, del_col, sets)
    
    av_item = average_item(R) #average rating for each item

    #validate the model on the training set
    i, j = np.where(R != 0)
    err = np.sum((R[i, j] - av_item[j])**2)   #index j because you are looking at movies 
    N = len(R[i, j])
    rmse = np.sqrt(err / N)
    print('The RMSE is:', rmse)
    err_abs = abs(R[i, j] - av_item[j]) 
    mas = np.sum(err_abs) / N
    print('The MAS is:', mas)
    RMSE.append(rmse)
    MAS.append(mas)
     
    #validate the model on the test set
    i, j = np.where(R_test != 0)
    err = np.sum((R_test[i, j] - av_item[j])**2)   
    N = len(R_test[i, j])
    rmse = np.sqrt(err / N)
    print('The RMSE is:', rmse)
    err_abs = abs(R_test[i, j] - av_item[j]) 
    mas = np.sum(err_abs) / N
    print('The MAS is:', mas)
    RMSE_test.append(rmse)
    MAS_test.append(mas)
    
print('==================================')
print('Average RMSE score on the training set:', sum(RMSE) / 5)
print('Average MAS score on the training set:', sum(MAS) / 5)
print('Average RMSE score on the test set:', sum(RMSE_test) / 5)
print('Average MAS score on the test set:', sum(MAS_test) / 5)

=== Fold 1 ===
The RMSE is: 0.9745077572164047
The MAS is: 0.7785339077439061
The RMSE is: 0.9782885041147636
The MAS is: 0.782240064947386
=== Fold 2 ===
The RMSE is: 0.9742110511055537
The MAS is: 0.7784146337085104
The RMSE is: 0.979261316276066
The MAS is: 0.7814255097637235
=== Fold 3 ===
The RMSE is: 0.9741355059818755
The MAS is: 0.7783057193728362
The RMSE is: 0.9798385393452186
The MAS is: 0.7825134845241571
=== Fold 4 ===
The RMSE is: 0.9741352904134655
The MAS is: 0.77814617255234
The RMSE is: 0.9797023081383784
The MAS is: 0.7830687328869811
=== Fold 5 ===
The RMSE is: 0.9741720595435537
The MAS is: 0.7782838112767951
The RMSE is: 0.9794528522276875
The MAS is: 0.7821295188494366
Average RMSE score on the training set: 0.9742323328521705
Average MAS score on the training set: 0.7783368489308776
Average RMSE score on the test set: 0.9793087040204229
Average MAS score on the test set: 0.7822754621943369


#### Average user rating recommender system

In [32]:
def average_user(R):
    #function which computes the average rating for each user (i.e average value of each row)
    av = np.sum(R) / np.count_nonzero(R)   #the fallback value we use when rows are completely empty in the training set
    
    #we are computing the average rating of each user now
    row_sums = np.sum(R, axis = 1) #we sum together rows
    row_n0_count =  np.count_nonzero(R, axis = 1) #number of non-zero ratings along each row
    i_miss = np.where(row_sums == 0) #users for which there are no ratings in the training set
    row_sums[i_miss] = av   #for users which went missing in the set, we use the fallback value av, i.e we assume that their average rating is the global average
    row_n0_count[i_miss] = 1

    av_user = row_sums / row_n0_count  #average rating for each user
    return av_user

In [33]:
#average rating per user approach

indx = np.arange(1, 6, 1)
RMSE = []
MAS = []
RMSE_test = []  
MAS_test = []

for i in range(5):   #do a loop for the 5-fold validation
    print('=== Fold {} ==='.format(i + 1))
    train_index = np.delete(indx, i)  
    test_index = i + 1               
    R, R_test = ut_matrices(train_index, test_index, df, del_col, sets)
    
    av_user = average_user(R)  #average rating for each user
    
    #validate the model on the training set
    i, j = np.where(R != 0)
    err = np.sum((R[i, j] - av_user[i])**2)   #index i because you are looking at users 
    N = len(R[i, j])
    rmse = np.sqrt(err / N)
    print('The RMSE is:', rmse)
    err_abs = abs(R[i, j] - av_user[i]) 
    mas = np.sum(err_abs) / N
    print('The MAS is:', mas)
    RMSE.append(rmse)
    MAS.append(mas)
    
    
    #validate the model on the test set
    i, j = np.where(R_test != 0)
    err = np.sum((R_test[i, j] - av_user[i])**2)   #index j because you are looking at movies (j is the x-axis)
    N = len(R_test[i, j])
    rmse = np.sqrt(err / N)
    print('The RMSE is:', rmse)
    err_abs = abs(R_test[i, j] - av_user[i]) 
    mas = np.sum(err_abs) / N
    print('The MAS is:', mas)
    RMSE_test.append(rmse)
    MAS_test.append(mas)
    
print('==================================')
print('Average RMSE score on the training set:', sum(RMSE) / 5)
print('Average MAS score on the training set:', sum(MAS) / 5)
print('Average RMSE score on the test set:', sum(RMSE_test) / 5)
print('Average MAS score on the test set:', sum(MAS_test) / 5)

=== Fold 1 ===
The RMSE is: 1.0282112145879772
The MAS is: 0.8231646937330492
The RMSE is: 1.0332851148511624
The MAS is: 0.8276196434496766
=== Fold 2 ===
The RMSE is: 1.0279645608350645
The MAS is: 0.8230432891502796
The RMSE is: 1.0343202435655925
The MAS is: 0.8267121952182561
=== Fold 3 ===
The RMSE is: 1.027497537854964
The MAS is: 0.8225581428281584
The RMSE is: 1.0362066208951601
The MAS is: 0.8302512208718921
=== Fold 4 ===
The RMSE is: 1.0275705270145654
The MAS is: 0.8228490278473736
The RMSE is: 1.0359759146658403
The MAS is: 0.829131825337574
=== Fold 5 ===
The RMSE is: 1.0271069887203652
The MAS is: 0.8221218215901928
The RMSE is: 1.037752961611784
The MAS is: 0.831167149873029
Average RMSE score on the training set: 1.0276701658025873
Average MAS score on the training set: 0.8227473950298106
Average RMSE score on the test set: 1.035508171117908
Average MAS score on the test set: 0.8289764069500857


#### Linear regression model recommender systems
'Full' variant of linear regression model

In [41]:
#'full' variant of linear regression
from sklearn.linear_model import LinearRegression

indx = np.arange(1, 6, 1)
RMSE = []
MAS = []
RMSE_test = []  
MAS_test = []

for i in range(5):   #do a loop for the 5-fold validation
    print('=== Fold {} ==='.format(i + 1))
    train_index = np.delete(indx, i)  
    test_index = i + 1               
    R, R_test = ut_matrices(train_index, test_index, df, del_col, sets)
    
    #find av_user (R_user) first
    av_user = average_user(R)  #average rating for each user
    
    #find av_item (R_item) next
    av_item = average_item(R)
    
    #create X matrix, the input matrix for the linear regression. This is a Nx2 matrix where each pair is the corresponding [av_user, av_item]
    X = np.zeros((R.shape[0] * R.shape[1], 2))
    i = 0 
    for user in av_user:
        for item in av_item:
            X[i] = [user, item]
            i += 1
    
    #create Y matrix. This is the matrix we want to fit, it is the utility matrix but flattened
    Y = R.flatten()
    n0 = np.where(Y != 0)[0] #these are the movie/user pairs for which we actually have data, this is what we will feed the model for training
    
    #the linear regression model
    reg = LinearRegression(fit_intercept = True).fit(X[n0], Y[n0])
    
    
    #make the prediction on the training set
    n0 = np.where(Y != 0)[0]  #ratings we have in the training set
    y_pred = reg.predict(X[n0])  #make the prediction
    y_pred[y_pred<1], y_pred[y_pred>5] = 1, 5  #rounding predictions that are >5 and <1
    N = len(y_pred)
    err = np.sum((Y[n0] - y_pred)**2)
    rmse = np.sqrt(err / N)
    err_abs = abs(Y[n0] - y_pred)
    mas = np.sum(err_abs) / N

    print('The RMSE on the training set is:', rmse)
    print('The MAS on training set is:', mas)
    RMSE.append(rmse)
    MAS.append(mas)
    
    
    #make the prediction on the test set
    Y_test = R_test.flatten()
    n0_test = np.where(Y_test != 0)[0]  #ratings we have in the test set
    y_pred = reg.predict(X[n0_test])  #make the prediction
    y_pred[y_pred<1], y_pred[y_pred>5] = 1, 5  #rounding predictions that are >5 and <1
    N = len(y_pred)
    err = np.sum((Y_test[n0_test] - y_pred)**2)
    rmse = np.sqrt(err / N)
    err_abs = abs(Y_test[n0_test] - y_pred)
    mas = np.sum(err_abs) / N

    print('The RMSE on the test set is:', rmse)
    print('The MAS on test set is:', mas)
    RMSE_test.append(rmse)
    MAS_test.append(mas)
    
print('==================================')
print('Average RMSE score on the training set:', sum(RMSE) / 5)
print('Average MAS score on the training set:', sum(MAS) / 5)
print('Average RMSE score on the test set:', sum(RMSE_test) / 5)
print('Average MAS score on the test set:', sum(MAS_test) / 5)
    
    


=== Fold 1 ===
The RMSE on the training set is: 0.9148519613209415
The MAS on training set is: 0.7249154209868554
The RMSE on the test set is: 0.922274063221647
The MAS on test set is: 0.7315643557542193
=== Fold 2 ===
The RMSE on the training set is: 0.914598363676717
The MAS on training set is: 0.7250973539542932
The RMSE on the test set is: 0.924146545163293
The MAS on test set is: 0.731023996831165
=== Fold 3 ===
The RMSE on the training set is: 0.914508959721135
The MAS on training set is: 0.7248020357356075
The RMSE on the test set is: 0.9248493509816402
The MAS on test set is: 0.7331803236564802
=== Fold 4 ===
The RMSE on the training set is: 0.9142522704434873
The MAS on training set is: 0.7243854881063938
The RMSE on the test set is: 0.9252130436375828
The MAS on test set is: 0.7337647160677788
=== Fold 5 ===
The RMSE on the training set is: 0.9146073973517558
The MAS on training set is: 0.7248730640916484
The RMSE on the test set is: 0.9246036745610281
The MAS on test set is:

'Simplified' variant of linear regression model

In [42]:
#'simplified' variant of linear regression
#exact same cell than the previous but the argument fit_intercept in LinearRegression is set to False

indx = np.arange(1, 6, 1)
RMSE = []
MAS = []
RMSE_test = []  
MAS_test = []

for i in range(5):   #do a loop for the 5-fold validation
    print('=== Fold {} ==='.format(i + 1))
    train_index = np.delete(indx, i)  
    test_index = i + 1               
    R, R_test = ut_matrices(train_index, test_index, df, del_col, sets)
    
    #find av_user (R_user) first
    av_user = average_user(R)  #average rating for each user
    
    #find av_item (R_item) next
    av_item = average_item(R)
    
    #create X matrix, the input matrix for the linear regression. This is a Nx2 matrix where each pair is the corresponding [av_user, av_item]
    X = np.zeros((R.shape[0] * R.shape[1], 2))
    i = 0 
    for user in av_user:
        for item in av_item:
            X[i] = [user, item]
            i += 1
    
    #create Y matrix. This is the matrix we want to fit, it is the utility matrix but flattened
    Y = R.flatten()
    n0 = np.where(Y != 0)[0] #these are the movie/user pairs for which we actually have data, this is what we will feed the model for training
    
    #the linear regression model
    reg = LinearRegression(fit_intercept = False).fit(X[n0], Y[n0])
    
    
    #make the prediction on the training set
    n0 = np.where(Y != 0)[0]  #ratings we have in the training set
    y_pred = reg.predict(X[n0])  #make the prediction
    y_pred[y_pred<1], y_pred[y_pred>5] = 1, 5  #rounding predictions that are >5 and <1
    N = len(y_pred)
    err = np.sum((Y[n0] - y_pred)**2)
    rmse = np.sqrt(err / N)
    err_abs = abs(Y[n0] - y_pred)
    mas = np.sum(err_abs) / N

    print('The RMSE on the training set is:', rmse)
    print('The MAS on training set is:', mas)
    RMSE.append(rmse)
    MAS.append(mas)
    
    
    #make the prediction on the test set
    Y_test = R_test.flatten()
    n0_test = np.where(Y_test != 0)[0]  #ratings we have in the test set
    y_pred = reg.predict(X[n0_test])  #make the prediction
    y_pred[y_pred<1], y_pred[y_pred>5] = 1, 5  #rounding predictions that are >5 and <1
    N = len(y_pred)
    err = np.sum((Y_test[n0_test] - y_pred)**2)
    rmse = np.sqrt(err / N)
    err_abs = abs(Y_test[n0_test] - y_pred)
    mas = np.sum(err_abs) / N

    print('The RMSE on the test set is:', rmse)
    print('The MAS on test set is:', mas)
    RMSE_test.append(rmse)
    MAS_test.append(mas)
    
print('==================================')
print('Average RMSE score on the training set:', sum(RMSE) / 5)
print('Average MAS score on the training set:', sum(MAS) / 5)
print('Average RMSE score on the test set:', sum(RMSE_test) / 5)
print('Average MAS score on the test set:', sum(MAS_test) / 5)

=== Fold 1 ===
The RMSE on the training set is: 0.9469112273006468
The MAS on training set is: 0.7586850416285784
The RMSE on the test set is: 0.9509045262076379
The MAS on test set is: 0.7624355491082995
=== Fold 2 ===
The RMSE on the training set is: 0.946613621025228
The MAS on training set is: 0.7588075580587107
The RMSE on the test set is: 0.9518866532928942
The MAS on test set is: 0.761621008744909
=== Fold 3 ===
The RMSE on the training set is: 0.9464833916507313
The MAS on training set is: 0.7585399231684952
The RMSE on the test set is: 0.9529051818110343
The MAS on test set is: 0.7636643502018701
=== Fold 4 ===
The RMSE on the training set is: 0.9463686782920321
The MAS on training set is: 0.7582463628435816
The RMSE on the test set is: 0.952863944272195
The MAS on test set is: 0.7639724366523194
=== Fold 5 ===
The RMSE on the training set is: 0.9464614186441606
The MAS on training set is: 0.7585352330518261
The RMSE on the test set is: 0.953375742463524
The MAS on test set is